ENTRENAMIENTO CON OCT

# Xception

In [ ]:
import tensorflow as tf
tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
import tensorflow as tf
tf.__version__

## Carga de Datos

In [ ]:
from Modelos_TL.load_data import load_data
import tensorflow as tf

base_dir='efs2/OCT2017'     #poner path de OCT2017
model_name= 'Xception'
IMG_SIZE = (150, 150)
#train_dataset, test_dataset, validation_dataset, class_names= load_data(base_dir, IMG_SIZE)

In [ ]:
print(class_names)

## Crear el modelo

In [ ]:
from Modelos_TL.make_model import modelo
import tensorflow as tf
import os

# ENTRENAMIENTO CON OCT PARA METODOLOGIAS 2 Y 4
# Xception

IMG_SIZE = (150, 150)
metrics = [tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'),
           tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc'),
           tf.keras.metrics.AUC(name='prc', curve='PR'),
           ]
         

In [ ]:
model, base_model = modelo(IMG_SIZE, 'Xception', metrics)

In [ ]:
train_dataset, test_dataset, validation_dataset, class_names= load_data(base_dir, IMG_SIZE)
model.evaluate(test_dataset)

## Checkpoint

In [ ]:
checkpoint_path = "efs/Checkpoints Xception/Entrenamiento con OCT/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)


## Congelo el modelo base

In [ ]:
from Modelos_TL.Train_model import train, fine_tunning
base_model.trainable=False
print(len(model.trainable_variables))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)

## Entrenamiento capas superiores

In [ ]:
model.summary()

In [ ]:
history = train(model, base_dir, epoch=15, cp_callback=cp_callback, IMG_SIZE=IMG_SIZE)

In [ ]:
# Restore the weights
#model.load_weights(checkpoint_path)
import pandas as pd

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

## Visualizar metricas

In [ ]:
from utils import *
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np

fine_tunning=False
history_fine=None
BATCH_SIZE=32
baseline_results = model.evaluate(test_dataset,
                                  batch_size=32, verbose=0)


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
metrics = ['loss', 'prc', 'precision', 'recall']


image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model.predict(image_batch_test, batch_size=BATCH_SIZE)


### Matriz de Confusión 

In [ ]:

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)


### ROC

In [ ]:

plot_roc("Train", train_labels, train_predictions, color=colors[0])
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()


### PRC

In [ ]:

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

### Metricas

In [ ]:
plot_metrics(history, fine_tunning, history_fine)

plt.show()

## Fine tunning

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
base_model.trainable=True
len(model.trainable_variables)

In [ ]:
fine_tune_at=90

In [ ]:
model.trainable = True
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(len(model.trainable_variables))
print('start training')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)

## Fine-Tunning capas superiores

In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def fine_tunning(modelo, base_dir, total_epoch, cp_callback, IMG_SIZE):
    train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE)
    
    start_time = time.time()
    # Training the model
    history_fine = modelo.fit(train_dataset,
                              epochs=total_epoch,
                              validation_data=validation_dataset,
                              callbacks=[cp_callback],
                              initial_epoch=15)

    end_time = time.time()
    print("--- Time taken to train : %s hours ---" % ((end_time - start_time) // 3600))
    # Freeze all the layers before the `fine_tune_at` layer

    return history_fine


In [ ]:
total_epoch=30
history_fine = fine_tunning(model, base_dir, total_epoch, cp_callback, IMG_SIZE)

In [ ]:
model.save('efs/Checkpoints Xception/Modelo entrenado con OCT/Xception_OCT')

In [ ]:
history.epoch[-1]

### Visualizar metricas

In [ ]:
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np
from utils import *

In [ ]:
train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE)

image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model.predict(image_batch_test, batch_size=BATCH_SIZE)

baseline_results = model.evaluate(test_dataset,
                                  batch_size=32, verbose=0)

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)

plot_roc("Train", train_labels, train_predictions, color='red')
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_metrics(history, fine_tunning, history_fine)

In [ ]:
from utils import visualize

visualize(history, baseline_results, modelo, train_labels, test_labels, test_predictions, train_predictions,
              fine_tunning=False, history_fine=None)

# MobileNet

## Crear el modelo

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus, 'GPU')

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
from Modelos_TL.make_model import modelo
from Modelos_TL.load_data import load_data
import tensorflow as tf
import os

# ENTRENAMIENTO CON OCT PARA METODOLOGIAS 2 Y 4

base_dir='efs2/OCT2017' 
IMG_SIZE = (224, 224)
with strategy.scope():
    metrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc'),
               tf.keras.metrics.AUC(name='prc', curve='PR'),
               ]
train_dataset, test_dataset, validation_dataset, class_names= load_data(base_dir, IMG_SIZE, strategy)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
model_MN, base_model_MN = modelo(IMG_SIZE, 'MobileNetV2', metrics, strategy)

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
model_MN.evaluate(test_dataset)

## Checkpoint

In [ ]:
checkpoint_path = "efs/Checkpoints Mobilenet/Entrenamiento con OCT2017/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)


## Congelo el modelo base

In [ ]:
from Modelos_TL.Train_model import train, fine_tunning
base_model_MN.trainable=False
print(len(model_MN.trainable_variables))
with strategy.scope():
    model_MN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=metrics)
model_MN.summary()

## Entrenamiento capas superiores

In [ ]:
history_MN = train(model_MN, base_dir, epoch=3, cp_callback=cp_callback, IMG_SIZE=IMG_SIZE, strategy=strategy)

In [ ]:
import pandas as pd

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_MN.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

### Visualizar metricas

In [ ]:
from utils import *
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np

fine_tunning=False
history_fine=None
baseline_results = model_MN.evaluate(test_dataset,
                                  batch_size=32, verbose=0)


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
metrics = ['loss', 'prc', 'precision', 'recall']


image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model_MN.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model_MN.predict(image_batch_test, batch_size=BATCH_SIZE)


### Matriz de Confusión 

In [ ]:

for name, value in zip(model_MN.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)


### ROC

In [ ]:

plot_roc("Train", train_labels, train_predictions, color=colors[0])
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()


### PRC

In [ ]:

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

### Metricas

In [ ]:
plot_metrics(history_MN, fine_tunning, history_fine)

plt.show()

## Fine tunning

In [ ]:
with strategy.scope():
    model_MN.load_weights(checkpoint_path)

In [ ]:
base_model_MN.trainable=True
len(model_MN.trainable_variables)

In [ ]:
fine_tune_at=90

In [ ]:
model_MN.trainable = True
base_model_MN.trainable = True
for layer in base_model_MN.layers[:fine_tune_at]:
    layer.trainable = False

print(len(model_MN.trainable_variables))

In [ ]:
model_MN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)

In [ ]:
from Modelos_TL.Train_model import train, fine_tunning

In [ ]:
history_MN.epoch[-1]

## Fine-Tunning capas superiores

In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from Modelos_TL.load_data import load_data

def fine_tunning(modelo, base_dir, total_epoch, history, cp_callback, IMG_SIZE, strategy):
    train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE, strategy)
    
    start_time = time.time()
    # Training the model
    history_fine = modelo.fit(train_dataset,
                              epochs=total_epoch,
                              validation_data=validation_dataset,
                              callbacks=[cp_callback],
                              initial_epoch=history.epoch[-1])

    end_time = time.time()
    print("--- Time taken to train : %s hours ---" % ((end_time - start_time) // 3600))
    # Freeze all the layers before the `fine_tune_at` layer

    return history_fine


total_epoch=16
history_fine = fine_tunning(model_MN, base_dir, total_epoch, history_MN, cp_callback, IMG_SIZE, strategy)

In [ ]:
model_MN.save('efs2/Checkpoints Mobilenet/Modelo entrenado con OCT/Mobilenet_OCT.h5')

In [ ]:
history_fine.epoch[29]

## Visualizar metricas

In [ ]:
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np
from utils import *

In [ ]:

image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model_MN.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model_MN.predict(image_batch_test, batch_size=BATCH_SIZE)

baseline_results = model_MN.evaluate(test_dataset,
                                  batch_size=32, verbose=0)

for name, value in zip(model_MN.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)

plot_roc("Train", train_labels, train_predictions, color='red')
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_metrics(history_MN, fine_tunning, history_fine)

In [ ]:
model_MN.save('efs/Checkpoints Mobilenet/Entrenamiento con OCT2017/MobileNet_OCT')

In [ ]:
!pip install tensorflow==2.3

In [ ]:
from utils import visualize

visualize(history, baseline_results, modelo, train_labels, test_labels, test_predictions, train_predictions,
              fine_tunning=False, history_fine=None)

# Resnet

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  8


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus, 'GPU')

In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


## Crear el modelo

In [18]:
from Modelos_TL.make_model import modelo
import tensorflow as tf
import os

# ENTRENAMIENTO CON OCT PARA METODOLOGIAS 2 Y 4
base_dir='efs2/OCT2017' 
IMG_SIZE = (150, 150)
with strategy.scope():
    metrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc'),
               tf.keras.metrics.AUC(name='prc', curve='PR'),
               ]

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [19]:
with strategy.scope():
    model, base_model = modelo(IMG_SIZE, 'ResNet50', metrics, strategy)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 150, 150, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 150, 150, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_1 (Tenso [(None, 150, 150, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 5, 5, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)             

In [ ]:
model.evaluate(test_dataset)

## Checkpoint

In [20]:
checkpoint_path = "efs2/Checkpoints ResNet50/Entrenamiento con OCT/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)


## Congelo el modelo base

In [21]:
from Modelos_TL.Train_model import train, fine_tunning
base_model.trainable=False
print(len(model.trainable_variables))
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=metrics)

4


## Entrenamiento capas superiores

In [ ]:
model.summary()

In [16]:
epoch=15
history = train(model, base_dir, epoch, cp_callback, IMG_SIZE, strategy)

Found 72136 files belonging to 2 classes.
Found 726 files belonging to 2 classes.
start training
Epoch 1/15
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
508/508 [==============================] - ETA: 0s - loss: 0.2443 - tp: 20593.0000 - fp: 3508.0000 - tn: 37713.0000 - fn: 3154.0000 - accuracy: 0.8975 - precision: 0.8544 - recall: 0.8672 - auc: 0.9604 - prc: 0.9215
Epoch 00001: val_accuracy improved from -inf to 0.98072, saving model to efs2/Checkpoints ResNet50/Entrenamiento con OCT/cp.ckpt
508/508 [==============================] - 311s 613ms/step - loss: 0.2443 - tp: 20593.0000 - fp: 3508.0000 - tn: 37713.0000 - fn: 3154.0000 - accuracy: 0.8975 - precision: 0.8544 - recall: 0.8672 - auc: 0.9604 - prc: 0.9215 - val_loss: 0.0927 - val_tp: 238.0000 - val_fp: 10.0000 - val_tn: 474.0

In [22]:
# Restore the weights
checkpoint_path = "efs2/Checkpoints ResNet50/Entrenamiento con OCT/cp.ckpt"
with strategy.scope():
    model.load_weights(checkpoint_path)

## Visualizar metricas

In [ ]:
from utils import *
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np

fine_tunning=False
history_fine=None
BATCH_SIZE=32
baseline_results = model.evaluate(test_dataset,
                                  batch_size=32, verbose=0)


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
metrics = ['loss', 'prc', 'precision', 'recall']


image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model.predict(image_batch_test, batch_size=BATCH_SIZE)


### Matriz de Confusión 

In [ ]:

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)


### ROC

In [ ]:

plot_roc("Train", train_labels, train_predictions, color=colors[0])
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()


### PRC

In [ ]:

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

### Metricas

In [ ]:
plot_metrics(history, fine_tunning, history_fine)

plt.show()

## Fine tunning

In [24]:
base_model.trainable=True
len(model.trainable_variables)

216

In [25]:
fine_tune_at=90

In [26]:
model.trainable = True
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(len(model.trainable_variables))
print('start training')

106
start training


In [27]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)

In [30]:
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from Modelos_TL.load_data import load_data



def fine_tunningg(modelo, base_dir, total_epoch, history, cp_callback, IMG_SIZE, strategy):
    train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE, strategy)
    
    start_time = time.time()
    # Training the model
    history_fine = modelo.fit(train_dataset,
                              epochs=total_epoch,
                              validation_data=validation_dataset,
                              callbacks=[cp_callback],
                              initial_epoch=history)

    end_time = time.time()
    print("--- Time taken to train : %s hours ---" % ((end_time - start_time) // 3600))
    # Freeze all the layers before the `fine_tune_at` layer

    return history_fine



## Fine-Tunning capas superiores

In [31]:
total_epoch=30
from Modelos_TL.Train_model import train, fine_tunning
history_fine = fine_tunningg(model, base_dir, total_epoch, 15, cp_callback, IMG_SIZE, strategy)

Found 72136 files belonging to 2 classes.
Found 726 files belonging to 2 classes.
Epoch 16/30
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 106 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 106 all-reduces with algorithm = nccl, num_packs = 1
508/508 [==============================] - ETA: 0s - loss: 0.1178 - tp: 22812.0000 - fp: 1814.0000 - tn: 39428.0000 - fn: 914.0000 - accuracy: 0.9580 - precision: 0.9263 - recall: 0.9615 - auc: 0.9888 - prc: 0.9750
Epoch 00016: val_accuracy improved from -inf to 0.99036, saving model to efs2/Checkpoints ResNet50/Entrenamiento con OCT/cp.ckpt
508/508 [==============================] - 516s 1s/step - loss: 0.1178 - tp: 22812.0000 - fp: 1814.0000 - tn: 39428.0000 - fn: 914.0000 - accuracy: 0.9580 - precision: 0.9263 - recall: 0.9615 - auc: 0.9888 - prc: 0.9750 - val_loss: 0.0266 - val_tp: 239.0000 - val_fp: 4.0000 - val_tn: 480.0000 - val_fn: 3.

In [32]:
with strategy.scope():
    model.save('efs2/Checkpoints ResNet50/modelo entrenado con OCT/ResNet50_OCT.h5')

### Visualizar metricas

In [ ]:
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np
from utils import *

In [ ]:
train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE)

image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model.predict(image_batch_test, batch_size=BATCH_SIZE)

baseline_results = model.evaluate(test_dataset,
                                  batch_size=32, verbose=0)

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)

plot_roc("Train", train_labels, train_predictions, color='red')
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

plot_metrics(history, fine_tunning, history_fine)

In [ ]:
from utils import visualize

visualize(history, baseline_results, modelo, train_labels, test_labels, test_predictions, train_predictions,
              fine_tunning=False, history_fine=None)

# InceptionV3

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  8


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus, 'GPU')

In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


## Crear el modelo

In [4]:
from Modelos_TL.make_model import modelo
import tensorflow as tf
import os

# ENTRENAMIENTO CON OCT PARA METODOLOGIAS 2 Y 4
base_dir='efs2/OCT2017' 
IMG_SIZE = (150, 150)
with strategy.scope():
    metrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc'),
               tf.keras.metrics.AUC(name='prc', curve='PR'),
               ]
    model, base_model = modelo(IMG_SIZE, 'Inception', metrics, strategy)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
model.evaluate(test_dataset)

## Checkpoint

In [6]:
!mkdir 'efs2/Checkpoints Inception'
!mkdir 'efs2/Checkpoints Inception/Entrenamiento con OCT'

In [7]:
checkpoint_path = "efs2/Checkpoints Inception/Entrenamiento con OCT/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)


## Congelo el modelo base

In [8]:
from Modelos_TL.Train_model import train, fine_tunning
base_model.trainable=False
print(len(model.trainable_variables))
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=metrics)

4


## Entrenamiento capas superiores

In [ ]:
model.summary()

In [ ]:
epoch=15
history = train(model, base_dir, epoch, cp_callback, IMG_SIZE, strategy)

In [ ]:
# Restore the weights
checkpoint_path = "efs2/Checkpoints ResNet50/Entrenamiento con OCT/cp.ckpt"
with strategy.scope():
    model.load_weights(checkpoint_path)

## Visualizar metricas

In [ ]:
from utils import *
import tensorflow as tf
import keras
from keras import callbacks
from Modelos_TL.load_data import load_data


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np

fine_tunning=False
history_fine=None
BATCH_SIZE=32
baseline_results = model.evaluate(test_dataset,
                                  batch_size=32, verbose=0)


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
metrics = ['loss', 'prc', 'precision', 'recall']


image_batch_train, train_labels = next(iter(train_dataset))
image_batch_test, test_labels = next(iter(test_dataset))

train_predictions = model.predict(image_batch_train, batch_size=BATCH_SIZE)
test_predictions = model.predict(image_batch_test, batch_size=BATCH_SIZE)


### Matriz de Confusión 

In [ ]:

for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions)


### ROC

In [ ]:

plot_roc("Train", train_labels, train_predictions, color=colors[0])
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()


### PRC

In [ ]:

plot_prc("Train", train_labels, train_predictions, color=colors[0])
plot_prc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')
plt.legend(loc='lower right')
plt.show()

### Metricas

In [ ]:
plot_metrics(history, fine_tunning, history_fine)

plt.show()

## Fine tunning

In [ ]:
base_model.trainable=True
len(model.trainable_variables)

In [ ]:
fine_tune_at=90

In [ ]:
model.trainable = True
base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(len(model.trainable_variables))
print('start training')

In [ ]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)

In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from Modelos_TL.load_data import load_data



def fine_tunningg(modelo, base_dir, total_epoch, history, cp_callback, IMG_SIZE, strategy):
    train_dataset, test_dataset, validation_dataset, class_names = load_data(base_dir, IMG_SIZE, strategy)
    
    start_time = time.time()
    # Training the model
    history_fine = modelo.fit(train_dataset,
                              epochs=total_epoch,
                              validation_data=validation_dataset,
                              callbacks=[cp_callback],
                              initial_epoch=history)

    end_time = time.time()
    print("--- Time taken to train : %s hours ---" % ((end_time - start_time) // 3600))
    # Freeze all the layers before the `fine_tune_at` layer

    return history_fine



## Fine-Tunning capas superiores

In [ ]:
total_epoch=30
from Modelos_TL.Train_model import train, fine_tunning
history_fine = fine_tunningg(model, base_dir, total_epoch, 15, cp_callback, IMG_SIZE, strategy)

In [ ]:
with strategy.scope():
    model.save('efs2/Checkpoints ResNet50/modelo entrenado con OCT/ResNet50_OCT.h5')

# OpticNet

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus, 'GPU')

In [ ]:
gpus = tf.config.list_logical_devices('GPU')
tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy(gpus)

In [ ]:
import tensorflow as tf
from OpticNet.OpticNet71 import OpticNet
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Multiply, GlobalAveragePooling2D, Add, Dense, Activation, Maximum, ZeroPadding2D, \
    BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Lambda, UpSampling2D, \
    DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform

input_size= 160
num_of_classes= 2
with strategy.scope():
    model = OpticNet(input_size, num_of_classes,strategy)

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

## Cargo los Datos

In [ ]:
from OpticNet.load_data import load_data

In [ ]:
base_dir='efs2/OCT2017' 
train_batches, test_batches, val_batches, classes= load_data(base_dir, strategy)

In [ ]:
import os
checkpoint_path = "efs2/Checkpoints OpticNet/OpticNet_OCT_binary2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)

In [ ]:
with strategy.scope():
    model.load_weights("efs2/Checkpoints OpticNet/OpticNet_OCT_binary/cp.ckpt")

In [ ]:
from OpticNet.load_data import load_data
import tensorflow as tf
import time
from tensorflow.keras.optimizers import Adam


with strategy.scope():
    model.trainable=True
    model.compile(Adam(lr=.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from OpticNet.Train_Model import train
epoch= 2
history = train(model, base_dir, epoch, cb, strategy)

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model.save('my_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


BATCH_SIZE_PER_REPLICA = 16
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')
train_datagen = ImageDataGenerator(rotation_range=40,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           shear_range=0.2,
                           zoom_range=0.2,
                           rescale=1.0 / 255,
                           horizontal_flip=True,
                           fill_mode='nearest', 
                           validation_split=0.05)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
classes = ['NORMAL', 'AR']
train_batches = train_datagen.flow_from_directory(train_dir,
                                          target_size=(160, 160),
                                          classes=classes,
                                          subset='training',
                                          shuffle=True,
                                          batch_size=batch_size,
                                          class_mode='binary',
                                          color_mode='rgb')
test_batches = test_datagen.flow_from_directory(test_dir,
                                        target_size=(160, 160),
                                        classes=classes,
                                        shuffle=True,
                                        batch_size=batch_size,
                                        class_mode='binary',
                                        color_mode='rgb')
val_batches = train_datagen.flow_from_directory(train_dir,
                                       subset='validation',
                                       target_size=(160, 160),
                                       classes=classes,
                                       shuffle=True,
                                       batch_size=batch_size,
                                       class_mode='binary',
                                       color_mode='rgb')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Multiply, GlobalAveragePooling2D, Add, Dense, Activation, Maximum, ZeroPadding2D, \
    BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Lambda, UpSampling2D, \
    DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform


def res_conv(X, filters, base, s):
    name_base = base + '/branch'

    F1, F2, F3 = filters

    ##### Branch1 is the main path and Branch2 is the shortcut path #####

    X_shortcut = X

    ##### Branch1 #####
    # First component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    X = Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=name_base + '1/conv_1',
               kernel_initializer=glorot_uniform(seed=0))(X)

    # Second component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_2')(X)
    X = Activation('relu', name=name_base + '1/relu_2')(X)
    X = Conv2D(filters=F2, kernel_size=(2, 2), strides=(s, s), padding='same', name=name_base + '1/conv_2',
               kernel_initializer=glorot_uniform(seed=0))(X)

    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=name_base + '1/conv_3',
               kernel_initializer=glorot_uniform(seed=0))(X)

    ##### Branch2 ####
    X_shortcut = BatchNormalization(axis=-1, name=name_base + '2/bn_1')(X_shortcut)
    X_shortcut = Activation('relu', name=name_base + '2/relu_1')(X_shortcut)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=name_base + '2/conv_1',
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)

    # Final step: Add Branch1 and Branch2
    X = Add(name=base + '/Add')([X, X_shortcut])

    return X


def res_identity(X, filters, base):
    name_base = base + '/branch'

    F1, F2, F3 = filters

    ##### Branch1 is the main path and Branch2 is the shortcut path #####

    X_shortcut = X

    ##### Branch1 #####
    # First component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    Shortcut = Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=name_base + '1/conv_1',
               kernel_initializer=glorot_uniform(seed=0))(Shortcut)

    # Second component BranchOut 1
    X1 = BatchNormalization(axis=-1, name=name_base + '1/ConvBn_2')(X)
    X1 = Activation('relu', name=name_base + '1/ConvRelu_2')(X1)
    X1 = Conv2D(filters=F2, kernel_size=(2, 2), dilation_rate=(2, 2), strides=(1, 1), padding='same',
                name=name_base + '1/Conv_2', kernel_initializer=glorot_uniform(seed=0))(X1)

    # Second component BrancOut 2
    X2 = BatchNormalization(axis=-1, name=name_base + '1/SepBn_2')(X)
    X2 = Activation('relu', name=name_base + '1/SepRelu_2')(X2)
    X2 = SeparableConv2D(filters=F2, kernel_size=(2, 2), dilation_rate=(2, 2), strides=(1, 1), padding='same',
                         name=name_base + '1/SepConv_2', kernel_initializer=glorot_uniform(seed=0))(X2)

    # Second component Add-BranchOut
    X = Add(name=base + '/Add-2branches')([X1, X2])

    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=name_base + '1/conv_3',
               kernel_initializer=glorot_uniform(seed=0))(X)

    # Final step: Add Branch1 and the original Input itself
    X = Add(name=base + '/Add')([X_shortcut, X])

    return X


def EncoderDecoder(X, name_base):
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name=name_base + '/Downsample1')(X)
    # X = Conv2D(outgoing_depth, (2,2), strides=(1,1), dilation_rate=(2,2), padding='same', name = name_base +
    # '/DC1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = UpSampling2D(size=(2, 2), interpolation='bilinear', name=name_base + '/Upsample1')(X)
    X = Activation('sigmoid', name=name_base + '/Activate')(X)
    return X


def RDBI(X, filters, base, number):
    for i in range(number):
        X = res_identity(X, filters, base + '/id_' + str(1 + i))

    return X


def OpticNet(input_size, num_of_classes, strategy):
    input_shape = (input_size, input_size, 3)  # Height x Width x Channel
    X_input = Input(input_shape)
    X = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='CONV1', kernel_initializer=glorot_uniform(seed=0))(
        X_input)
    X = BatchNormalization(axis=-1, name='BN1')(X)
    X = Activation('relu', name='RELU1')(X)

    X = res_conv(X, [64, 64, 256], 'RC0', 1)

    # MID 1

    X1 = EncoderDecoder(X, 'EncoderDecoder1')

    X2 = RDBI(X, [32, 32, 256], 'RDBI1', 4)

    X = Multiply(name='Mutiply1')([X1, X2])

    X = Add(name='Add1')([X, X1, X2])

    X = res_conv(X, [128, 128, 512], 'RC1', 2)

    # MID 2

    X1 = EncoderDecoder(X, 'EncoderDecoder2')

    X2 = RDBI(X, [64, 64, 512], 'RDBI2', 4)

    X = Multiply(name='Mutiply2')([X1, X2])

    X = Add(name='Add2')([X, X1, X2])

    X = res_conv(X, [256, 256, 1024], 'RC2', 2)

    # MID 3

    X1 = EncoderDecoder(X, 'EncoderDecoder3')

    X2 = RDBI(X, [128, 128, 1024], 'RDBI3', 3)

    X = Multiply(name='Mutiply3')([X1, X2])

    X = Add(name='Add3')([X, X1, X2])

    X = res_conv(X, [512, 512, 2048], 'RC3', 2)

    # MID 4

    X1 = EncoderDecoder(X, 'EncoderDecoder4')

    X2 = RDBI(X, [256, 256, 2048], 'RDBI4', 3)

    X = Multiply(name='Mutiply4')([X1, X2])

    X = Add(name='Add4')([X, X1, X2])

    X = GlobalAveragePooling2D(name='global_avg_pool')(X)
    X = Dense(256, name='Dense_1')(X)
    X = Dense(num_of_classes, name='Dense_2')(X)
    X = Activation('softmax', name='classifier')(X)

    model = Model(inputs=X_input, outputs=X, name='')

    model.compile(Adam(learning_rate=.0001), loss='binary_crossentropy', metrics=['accuracy'])

    model.summary()

    return model

with strategy.scope():
    modelo= OpticNet(input_size, num_of_classes, strategy)

In [ ]:
 from OpticNet.load_data import load_data
import tensorflow as tf
import time
from tensorflow.keras.optimizers import Adam


with strategy.scope():
    modelo.load_weights('efs2/Checkpoints OpticNet/OpticNet_OCT/cp.ckpt')
    modelo.trainable=True
    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(model.layers))

    # Fine-tune from this layer onwards
    fine_tune_at = 274

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in modelo.layers[:fine_tune_at]:
        layer.trainable =  False
    modelo.compile(Adam(lr=.0001), loss='binary_crossentropy', metrics=['accuracy'])

history = modelo.fit(train_batches, 
                    shuffle=True,  
                    validation_data=test_batches, 
                    epochs=5, 
                    verbose=1, 
                    callbacks=[cb])

In [ ]:
with strategy.scope():
    modelo.trainable=True
    modelo.compile(Adam(lr=.00001), loss='binary_crossentropy', metrics=['accuracy'])

history = modelo.fit(train_batches, 
                    shuffle=True,  
                    validation_data=test_batches, 
                    epochs=5, 
                    verbose=1, 
                    callbacks=[cb])

In [ ]:
import os
import shutil

MN = os.listdir('efs/Checkpoints Mobilenet/Entrenamiento con OCT2017')
XC = os.listdir('efs/Checkpoints Xception/Entrenamiento con OCT')
RN = os.listdir('efs2/Checkpoints ResNet50/Entrenamiento con OCT')
for file in MN:
    src = 'efs/Checkpoints Mobilenet/Entrenamiento con OCT2017/'
    dest =  'Modelos_TL/Entrenamiento con OCT/Mobilenet/Checkpoints/'
    shutil.copy(src+file, dest+ file)
    
for file in XC:
    src = 'efs/Checkpoints Xception/Entrenamiento con OCT/'
    dest =  'Modelos_TL/Entrenamiento con OCT/Xception/Checkpoints/'
    shutil.copy(src+file, dest+ file)
    
for file in RN:
    src = 'efs2/Checkpoints ResNet50/Entrenamiento con OCT/'
    dest =  'Modelos_TL/Entrenamiento con OCT/Resnet/Checkpoints/'
    shutil.copy(src+file, dest+ file)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_datas(base_dir, strategy):
    BATCH_SIZE_PER_REPLICA = 16
    batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
    train_dir = os.path.join(base_dir, 'train')
    test_dir = os.path.join(base_dir, 'test')
    val_dir = os.path.join(base_dir, 'val')
    train_datagen = ImageDataGenerator(rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       rescale=1.0 / 255,
                                       horizontal_flip=True,
                                       fill_mode='nearest', 
                                       validation_split=0.1)
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)
    classes = ['NORMAL', 'AR']
    train_batches = train_datagen.flow_from_directory(train_dir,
                                                      target_size=(160, 160),
                                                      classes=classes,
                                                      subset='training',
                                                      shuffle=True,
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      color_mode='rgb')
    test_batches = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(160, 160),
                                                    classes=classes,
                                                    shuffle=True,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='rgb')
    val_batches = train_datagen.flow_from_directory(train_dir,
                                                   subset='validation',
                                                   target_size=(160, 160),
                                                   classes=classes,
                                                   shuffle=True,
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   color_mode='rgb')
    return train_batches, test_batches, val_batches, classes

train_batches, test_batches, val_batches, classes= load_datas(base_dir, strategy)

In [ ]:

with strategy.scope():
    model.trainable=True
    model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import time
from tensorflow.keras.optimizers import Adam


start_time = time.time()
# Training the model
batch_size=32
history2 = model.fit(train_batches,
                     shuffle=True,
                     validation_data=val_batches,
                     epochs=30,
                     verbose=1,
                     callbacks=cb,
                     initial_epoch=18)

end_time = time.time()
print("--- Time taken to train : %s hours ---" % ((end_time - start_time) // 3600))

In [ ]:
model.summary()